### Importando as bibliotecas necessárias

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import locale
import openpyxl
import re

#### Carregando o arquivo CSV em um DataFrame
#### Neste passo, carregaremos o arquivo CSV com os dados dos anúncios em um DataFrame do pandas
#### Certifique-se de ter o arquivo ANÁLISE-DE-MÉTRICAS-POR-ANÚNCIO-ÚLTIMOS-30-DIAS.csv no mesmo diretório deste notebook

In [15]:
dados_anuncios = pd.read_csv('ANÁLISE-DE-MÉTRICAS-POR-ANÚNCIO-ÚLTIMOS-30-DIAS.csv')

In [16]:
dados_anuncios.columns

Index(['Nome do anúncio', 'Alcance', 'Impressões', 'Frequência',
       'Valor usado (BRL)', 'Configuração de atribuição', 'Tipo de resultado',
       'Resultados', 'Custo por resultado', 'CPM (custo por 1.000 impressões)',
       'CTR (todos)', 'Classificação de qualidade',
       'Classificação da taxa de engajamento', 'Custo por lead',
       'Valor médio de conversão de compra',
       'Conversas por mensagem iniciadas',
       'Custo por conversa por mensagem iniciada',
       'CPC (custo por clique no link)', 'Cliques (todos)',
       'Engajamento com a Página', 'Reproduções de 25% do vídeo',
       'Reproduções de 50% do vídeo', 'Reproduções de 75% do vídeo',
       'Reproduções de 95% do vídeo', 'Reproduções de 100% do vídeo',
       'Início dos relatórios', 'Término dos relatórios'],
      dtype='object')

### Removendo linhas duplicadas
### Vamos remover duplicatas, pois elas podem distorcer as métricas e análises dos dados de anúncios

In [17]:
# Identifica e exibe as linhas duplicadas antes de removê-las
duplicatas = dados_anuncios[dados_anuncios.duplicated()]

# Verifica se existem duplicatas e exibe uma mensagem
if not duplicatas.empty:
    print("Linhas duplicadas encontradas e que serão removidas:")
    print(duplicatas)
else:
    print("Nenhuma linha duplicada encontrada.")

# Remove as linhas duplicadas
dados_anuncios = dados_anuncios.drop_duplicates()


Linhas duplicadas encontradas e que serão removidas:
                     Nome do anúncio  Alcance  Impressões  Frequência  \
27       AD 03 - CARROSSEL - 17 | 08        0           0         0.0   
29            AD 01 - IMAGEM STORIES        0           0         0.0   
32     AD 01 - CARROSSEL 1 - 18 | 04        0           0         0.0   
34                 AD 01 - CARROSSEL        0           0         0.0   
43    AD 01 - IMAGEM ÚNICA - 18 | 08        0           0         0.0   
44         AD 03 CARROSSEL - 06 | 10        0           0         0.0   
47    AD 02 - IMAGEM ÚNICA - 18 | 08        0           0         0.0   
48                 AD 01 - CARROSSEL        0           0         0.0   
50      AD 01 IMAGEM ÚNICA - 06 | 10        0           0         0.0   
51    AD 02 - IMAGEM ÚNICA - 18 | 08        0           0         0.0   
53            AD 02 - IMAGEM STORIES        0           0         0.0   
57     AD 01 - CARROSSEL 1 - 18 | 04        0           0         0.0  

### Removendo colunas vazias
### Aqui, removeremos colunas que estejam completamente vazias, pois não contribuem para a análise

In [18]:
dados_anuncios = dados_anuncios.dropna(axis=1, how='all')

In [19]:
# Função para extrair a data
def extrair_data(nome_anuncio):
    match = re.search(r'\d{2}/\d{2}/\d{4}', nome_anuncio)
    return match.group(0) if match else None

# Aplicando a função na coluna 'Nome do anúncio' e criando a nova coluna 'Data de criação'
dados_anuncios['Data de criação'] = dados_anuncios['Nome do anúncio'].apply(extrair_data)



### Exibindo os tipos de dados de cada coluna
### Essa inspeção é fundamental para verificar a consistência de tipos e ajustar a análise posteriormente, se necessário

In [20]:
dados_anuncios.dtypes

Nome do anúncio                          object
Alcance                                   int64
Impressões                                int64
Frequência                              float64
Valor usado (BRL)                       float64
Configuração de atribuição               object
Tipo de resultado                        object
Resultados                              float64
Custo por resultado                     float64
CPM (custo por 1.000 impressões)        float64
CTR (todos)                             float64
Classificação de qualidade               object
Classificação da taxa de engajamento     object
CPC (custo por clique no link)          float64
Cliques (todos)                           int64
Engajamento com a Página                float64
Reproduções de 25% do vídeo             float64
Reproduções de 50% do vídeo             float64
Reproduções de 75% do vídeo             float64
Reproduções de 95% do vídeo             float64
Reproduções de 100% do vídeo            

# Ajuste de tipos das colunas

In [21]:
# Configura o locale para o Brasil
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [22]:

# Função para formatar valores em moeda brasileira
def formatar_moeda(valor):
    return locale.currency(valor, grouping=True) if pd.notnull(valor) else valor

# Função para converter datas para o formato padrão e exibi-las no formato brasileiro
def converter_data(data):
    for fmt in ('%d/%m/%Y', '%Y-%m-%d'):
        try:
            data_formatada = pd.to_datetime(data, format=fmt)
            return data_formatada.strftime('%d/%m/%Y')  # Retorna em formato brasileiro
        except ValueError:
            continue
    return pd.NaT  # Retorna NaT se nenhum formato for adequado

# Aplicando as correções de tipos nas colunas conforme solicitado
dados_anuncios = dados_anuncios.astype({
    'Nome do anúncio': 'string',
    'Alcance': 'int64',
    'Impressões': 'int64',
    'Frequência': 'float64',
    'Configuração de atribuição': 'category',
    'Tipo de resultado': 'category',
    'Resultados': 'float64',
    'Classificação de qualidade': 'category',
    'Classificação da taxa de engajamento': 'category',
    'Cliques (todos)': 'int64',
    'Engajamento com a Página': 'float64',
    'Reproduções de 25% do vídeo': 'float64',
    'Reproduções de 50% do vídeo': 'float64',
    'Reproduções de 75% do vídeo': 'float64',
    'Reproduções de 95% do vídeo': 'float64',
    'Reproduções de 100% do vídeo': 'float64'
})

# Aplicando a função de formatação de moeda nas colunas de valores monetários
for coluna_monetaria in ['Valor usado (BRL)', 'Custo por resultado', 'CPM (custo por 1.000 impressões)', 'CPC (custo por clique no link)']:
    dados_anuncios[coluna_monetaria] = dados_anuncios[coluna_monetaria].apply(formatar_moeda)

# Convertendo datas para o formato brasileiro com a função converter_data
dados_anuncios['Início dos relatórios'] = dados_anuncios['Início dos relatórios'].apply(converter_data)
dados_anuncios['Término dos relatórios'] = dados_anuncios['Término dos relatórios'].apply(converter_data)

# Verificando datas nulas após a conversão
print("Datas nulas em 'Início dos relatórios':")
print(dados_anuncios[dados_anuncios['Início dos relatórios'].isnull()])

print("Datas nulas em 'Término dos relatórios':")
print(dados_anuncios[dados_anuncios['Término dos relatórios'].isnull()])

# Convertendo CTR para float64
dados_anuncios['CTR (todos)'] = pd.to_numeric(dados_anuncios['CTR (todos)'], errors='coerce')


Datas nulas em 'Início dos relatórios':
Empty DataFrame
Columns: [Nome do anúncio, Alcance, Impressões, Frequência, Valor usado (BRL), Configuração de atribuição, Tipo de resultado, Resultados, Custo por resultado, CPM (custo por 1.000 impressões), CTR (todos), Classificação de qualidade, Classificação da taxa de engajamento, CPC (custo por clique no link), Cliques (todos), Engajamento com a Página, Reproduções de 25% do vídeo, Reproduções de 50% do vídeo, Reproduções de 75% do vídeo, Reproduções de 95% do vídeo, Reproduções de 100% do vídeo, Início dos relatórios, Término dos relatórios, Data de criação]
Index: []

[0 rows x 24 columns]
Datas nulas em 'Término dos relatórios':
Empty DataFrame
Columns: [Nome do anúncio, Alcance, Impressões, Frequência, Valor usado (BRL), Configuração de atribuição, Tipo de resultado, Resultados, Custo por resultado, CPM (custo por 1.000 impressões), CTR (todos), Classificação de qualidade, Classificação da taxa de engajamento, CPC (custo por clique no 

In [23]:

# Exibir o dataframe atualizado e verificar os tipos das colunas
dados_anuncios.dtypes

Nome do anúncio                         string[python]
Alcance                                          int64
Impressões                                       int64
Frequência                                     float64
Valor usado (BRL)                               object
Configuração de atribuição                    category
Tipo de resultado                             category
Resultados                                     float64
Custo por resultado                             object
CPM (custo por 1.000 impressões)                object
CTR (todos)                                    float64
Classificação de qualidade                    category
Classificação da taxa de engajamento          category
CPC (custo por clique no link)                  object
Cliques (todos)                                  int64
Engajamento com a Página                       float64
Reproduções de 25% do vídeo                    float64
Reproduções de 50% do vídeo                    float64
Reproduçõe

In [24]:
# Identificar as linhas onde o valor de 'Alcance' é igual a zero
linhas_excluir = dados_anuncios[dados_anuncios['Alcance'] == 0]

# Extrair os nomes dos anúncios que serão excluídos
anuncios_excluidos = linhas_excluir['Nome do anúncio'].tolist()

# Imprimir os nomes dos anúncios que serão excluídos de forma mais bonita
print("Anúncios excluídos:")
for i, anuncio in enumerate(anuncios_excluidos, start=1):
    print(f"{i}. {anuncio}")

# Excluir as linhas onde 'Alcance' é igual a zero do DataFrame original
dados_anuncios = dados_anuncios[dados_anuncios['Alcance'] != 0]

# Resetar o índice após a exclusão
dados_anuncios.reset_index(drop=True, inplace=True)


Anúncios excluídos:
1. AD 04 - VÍDEO - 18 | 04
2. AD 03 - CARROSSEL 3  - 21 | 12
3. AD 02 - STORY - 05/06/2024
4. AD 03 - CARROSSEL ST - 24/05/2024
5. AD 04 - CARROSSEL 4  - 21 | 12
6. AD 01 - STORIES - 21 | 12
7. AD 01 - CARROSSEL FEED - 05/06/2024
8. AD 02 - IMAGEM STORIES
9. AD 03 CARROSSEL - 06 | 10
10. AD 03 - CARROSSEL - 17 | 08
11. AD 01 - IMAGEM ÚNICA - 14 | 07
12. AD 02 - CARROSSEL 2  - 21 | 12
13. AD 01 IMAGEM ÚNICA - 06 | 10
14. AD - 05 - CARROSSEL ST - 24/05/2024
15. AD 03 - IMAGEM ÚNICA - 18 | 04
16. AD 02 - CARROSSEL 1 - 18 | 04
17. AD 01 - CARROSSEL 1 - 18 | 04
18. AD 03 - CARROSSEL - 18 | 08
19. AD 01 - CARROSSEL - 17 | 02
20. AD 01 - IMAGEM STORIES
21. AD 01 - CARROSSEL
22. AD 01 - VÍDEO STORIES - 04/04/24
23. AD 01 - STORY - 05/06/2024
24. AD 01 - IMAGEM ÚNICA - 18 | 08
25. AD 04 - CARROSSEL ST - 24/05/2024
26. AD 02 - IMAGEM ÚNICA - 18 | 08
27. AD 02 - CARROSSEL ST - 24/05/2024
28. AD 03 - VÍDEO - 18 | 04
29. AD 05 - CARROSSEL ST - 24/05/2024
30. AD - 02 - CARROSSEL 

In [26]:
dados_anuncios.to_csv("miro_dados_ultimos_30_dias.csv", index=False)